# Mean Bias error

The current mlpack implementation is incorrect.

### Imports and installation of mlpack

In [0]:
%%capture
!sudo apt-get install libmlpack-dev 
import torch
import torch.nn as nn

### mlpack


#### CURRENT IMPLEMENTATION


In [0]:
%%capture
%%writefile test.cpp  

// This uses the computations present in mean_bias_error_impl.hpp in ann/loss_functions.
#include <iostream>
#include <armadillo>

using namespace std;
using namespace arma;

int main()
{
  // Constructor
  arma::mat x,y;
  arma::mat weight;
 
  x << -0.0494 << 1.6028 << 0.9639 << endr
    << -1.1958 << 0.0737 << 0.9648 << endr
    << -1.0486 << -0.7091 << 0.0745 << endr
    << -0.2121 << 0.8612 << 0.5924 << endr;

  y << 0.4316  << 0.5106 <<  0.7059 << endr
    << 0.0164  << 0.9255 << -0.8288 << endr
    << -0.4478 << 0.5571 << -0.0231 << endr
    << 1.1452  << 0.0864 << -1.0526 << endr;

  // Forward
  double loss = arma::accu(y - x) / y.n_cols;

  // Backward
  arma::mat output;
  output.set_size(arma::size(x));
  output.fill(-1.0);

  // Display
  cout << "------------------------------------------------------------------" << endl;
  cout << "USER-PROVIDED MATRICES : " << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "Input shape : "<< x.n_rows << " " << x.n_cols << endl;
  cout << "Input : " << endl << x << endl;
  cout << "Target shape : "<< y.n_rows << " " << y.n_cols << endl;
  cout << "Target : " << endl << y << endl;
  cout << "FORWARD : " << endl;
  cout << "Loss : \n" << loss << '\n';
  cout << "BACKWARD : " << endl;
  cout << "Output shape : "<< output.n_rows << " " << output.n_cols << endl;
  cout << "Output (sum) : " << endl << output << endl;
  cout << "Sum of all values in this matrix : " << arma::as_scalar(arma::accu(output)) << endl;
  return 0;
}

In [0]:
%%script bash
g++ test.cpp -o test -larmadillo && ./test

------------------------------------------------------------------
USER-PROVIDED MATRICES : 
------------------------------------------------------------------
Input shape : 4 3
Input : 
  -0.0494   1.6028   0.9639
  -1.1958   0.0737   0.9648
  -1.0486  -0.7091   0.0745
  -0.2121   0.8612   0.5924

Target shape : 4 3
Target : 
   0.4316   0.5106   0.7059
   0.0164   0.9255  -0.8288
  -0.4478   0.5571  -0.0231
   1.1452   0.0864  -1.0526

FORWARD : 
Loss : 
0.0360333
BACKWARD : 
Output shape : 4 3
Output (sum) : 
  -1.0000  -1.0000  -1.0000
  -1.0000  -1.0000  -1.0000
  -1.0000  -1.0000  -1.0000
  -1.0000  -1.0000  -1.0000

Sum of all values in this matrix : -12


#### NEW IMPLEMENTATION

In [0]:
%%capture
%%writefile test.cpp  

#include <iostream>
#include <armadillo>

using namespace std;
using namespace arma;

int main()
{
  // Constructor
  arma::mat x,y;
  arma::mat weight;
 
  x << -0.0494 << 1.6028 << 0.9639 << endr
    << -1.1958 << 0.0737 << 0.9648 << endr
    << -1.0486 << -0.7091 << 0.0745 << endr
    << -0.2121 << 0.8612 << 0.5924 << endr;

  y << 0.4316  << 0.5106 <<  0.7059 << endr
    << 0.0164  << 0.9255 << -0.8288 << endr
    << -0.4478 << 0.5571 << -0.0231 << endr
    << 1.1452  << 0.0864 << -1.0526 << endr;

  // Forward
  arma::mat loss_none = y - x;
  double loss_sum = arma::accu(loss_none);
  double loss_mean = loss_sum / x.n_elem;

  // Backward
  arma::mat output;
  output.set_size(arma::size(x));
  output.fill(-1.0);

  // Display
  cout << "------------------------------------------------------------------" << endl;
  cout << "USER-PROVIDED MATRICES : " << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "Input shape : "<< x.n_rows << " " << x.n_cols << endl;
  cout << "Input : " << endl << x << endl;
  cout << "Target shape : "<< y.n_rows << " " << y.n_cols << endl;
  cout << "Target : " << endl << y << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "SUM " << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "FORWARD : " << endl;
  cout << "Loss : \n" << loss_none << '\n';
  cout << "Loss (sum):\n" << loss_sum << '\n';
  cout << "BACKWARD : " << endl;
  cout << "Output shape : "<< output.n_rows << " " << output.n_cols << endl;
  cout << "Output (sum) : " << endl << output << endl;
  cout << "Sum of all values in this matrix : " << arma::as_scalar(arma::accu(output)) << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "MEAN " << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "FORWARD : " << endl;
  cout << "Loss (mean):\n" << loss_mean << '\n';
  cout << "BACKWARD : " << endl;
  cout << "Output shape : "<< output.n_rows << " " << output.n_cols << endl;
  cout << "Output (mean) : " << endl << output / x.n_elem << endl;
  cout << "Sum of all values in this matrix : " << arma::as_scalar(arma::accu(output / x.n_elem)) << endl;
  cout << "------------------------------------------------------------------" << endl;
  return 0;
}

In [0]:
%%script bash
g++ test.cpp -o test -larmadillo && ./test

------------------------------------------------------------------
USER-PROVIDED MATRICES : 
------------------------------------------------------------------
Input shape : 4 3
Input : 
  -0.0494   1.6028   0.9639
  -1.1958   0.0737   0.9648
  -1.0486  -0.7091   0.0745
  -0.2121   0.8612   0.5924

Target shape : 4 3
Target : 
   0.4316   0.5106   0.7059
   0.0164   0.9255  -0.8288
  -0.4478   0.5571  -0.0231
   1.1452   0.0864  -1.0526

------------------------------------------------------------------
SUM 
------------------------------------------------------------------
FORWARD : 
Loss : 
   0.4810  -1.0922  -0.2580
   1.2122   0.8518  -1.7936
   0.6008   1.2662  -0.0976
   1.3573  -0.7748  -1.6450

Loss (sum):
0.1081
BACKWARD : 
Output shape : 4 3
Output (sum) : 
  -1.0000  -1.0000  -1.0000
  -1.0000  -1.0000  -1.0000
  -1.0000  -1.0000  -1.0000
  -1.0000  -1.0000  -1.0000

Sum of all values in this matrix : -12
------------------------------------------------------------------
ME